In [68]:
import numpy as np
import cv2
import mediapipe as mp
import pandas as pd
import os
import math

In [69]:
mp_face_mes = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mes.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [70]:
images_wide = os.listdir('eye_wide') #Warning: in all images, BOTH eyes must be wide open
images_open = os.listdir('eye_open') #Warning: in all images, BOTH eyes must be open
images_narrowed = os.listdir('eye_narrowed') #Warning: in all images, BOTH eyes must be narrowed
images_closed = os.listdir('eye_closed') #Warning: in all images, BOTH eyes must be closed

print(len(images_wide), len(images_open), len(images_narrowed), len(images_closed))

909 1461 1448 1263


In [71]:
def update_mesh_points(image):
    try:
        frame = image.copy()
    except AttributeError:
        return None
    H, W, _ = frame.shape
    results_mesh = face_mesh.process(frame)
    if results_mesh.multi_face_landmarks:
        mesh_points = np.array([np.multiply([p.x, p.y, p.z], [W, H, max(W, H)]).astype(int) for p in results_mesh.multi_face_landmarks[0].landmark])
        return mesh_points
    return None

In [72]:
RIGHT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398, 362]
LEFT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246, 33]

def calculate_sclera_area(mesh_points):
    lex1, lex2 = mesh_points[33], mesh_points[133]
    rex1, rex2 = mesh_points[362], mesh_points[263]
    area_sclera_left = 0
    area_sclera_right = 0
    for i in range(len(LEFT_EYE) - 1):
        area_sclera_left += (mesh_points[LEFT_EYE[i], 0] * mesh_points[LEFT_EYE[i + 1], 1]) - (mesh_points[LEFT_EYE[i + 1], 0] * mesh_points[LEFT_EYE[i], 1])
    for i in range(len(RIGHT_EYE) - 1):
        area_sclera_right += (mesh_points[RIGHT_EYE[i], 0] * mesh_points[RIGHT_EYE[i + 1], 1]) - (mesh_points[RIGHT_EYE[i + 1], 0] * mesh_points[RIGHT_EYE[i], 1])
    area_sclera_left = abs(area_sclera_left / (2*(math.sqrt((lex1[0]-lex2[0])**2 + (lex1[1]-lex2[1])**2)**2)))
    area_sclera_right = abs(area_sclera_right / (2*(math.sqrt((rex1[0]-rex2[0])**2 + (rex1[1]-rex2[1])**2)**2)))
    return area_sclera_left, area_sclera_right

def calculate_width_over_height(mesh_points):
    lex1 = mesh_points[33]
    lex2 = mesh_points[133]
    rex1 = mesh_points[362]
    rex2 = mesh_points[263]
    ley1 = mesh_points[159]
    ley2 = mesh_points[145]
    rey1 = mesh_points[386]
    rey2 = mesh_points[374]
    try:
        reason_left = math.sqrt((lex1[0]-lex2[0])**2+(lex1[1]-lex2[1])**2+(lex1[2]-lex2[2])**2)/math.sqrt((ley1[0]-ley2[0])**2+(ley1[1]-ley2[1])**2+(ley1[2]-ley2[2])**2)
        if reason_left > 20: reason_left = 20
    except ZeroDivisionError:
        reason_left = 20
    try:
        reason_right = math.sqrt((rex1[0]-rex2[0])**2+(rex1[1]-rex2[1])**2+(rex1[2]-rex2[2])**2)/math.sqrt((rey1[0]-rey2[0])**2+(rey1[1]-rey2[1])**2+(rey1[2]-rey2[2])**2)
        if reason_right > 20: reason_right = 20
    except ZeroDivisionError:
        reason_right = 20
    return reason_left, reason_right

def get_features(frame):
    mesh_points = update_mesh_points(frame)
    if mesh_points is None:
        return None
    area_sclera_left, area_sclera_right = calculate_sclera_area(mesh_points)
    reason_left, reason_right = calculate_width_over_height(mesh_points)
    return (area_sclera_left, area_sclera_right, reason_left, reason_right)

In [73]:
def resize_if_too_small(photo):
    if photo.shape[0] < 300 and photo.shape[1] < 300:
        photo = cv2.resize(photo, (300, 300), interpolation = cv2.INTER_CUBIC)
    return photo

In [74]:
results_wide = []
results_open = []
results_narrowed = []
results_closed = []

print('Processing wide images...')
for image in images_wide:
    frame = cv2.imread('eye_wide/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_wide.append((result[0], result[2]))
        results_wide.append((result[1], result[3]))

print('Processing open images...')
for image in images_open:
    frame = cv2.imread('eye_open/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_open.append((result[0], result[2]))
        results_open.append((result[1], result[3]))

print('Processing narrowed images...')
for image in images_narrowed:
    frame = cv2.imread('eye_narrowed/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_narrowed.append((result[0], result[2]))
        results_narrowed.append((result[1], result[3]))

print('Processing closed images...')
for image in images_closed:
    frame = cv2.imread('eye_closed/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_closed.append((result[0], result[2]))
        results_closed.append((result[1], result[3]))

Processing wide images...


Processing open images...
Processing narrowed images...
Processing closed images...


In [75]:
print(len(results_wide), len(results_open), len(results_narrowed), len(results_closed))

1616 2740 1958 2460


In [76]:
results_wide = np.array(results_wide)
results_open = np.array(results_open)
results_narrowed = np.array(results_narrowed)
results_closed = np.array(results_closed)

In [77]:
df = pd.DataFrame(columns=['normalized_sclera_area', 'width_over_height', 'label'])

for value in results_wide:
    df.loc[len(df)] = {'normalized_sclera_area': value[0], 'width_over_height': value[1], 'label': 'wide'}
for value in results_open:
    df.loc[len(df)] = {'normalized_sclera_area': value[0], 'width_over_height': value[1], 'label': 'open'}
for value in results_narrowed:
    df.loc[len(df)] = {'normalized_sclera_area': value[0], 'width_over_height': value[1], 'label': 'narrowed'}
for value in results_closed:
    df.loc[len(df)] = {'normalized_sclera_area': value[0], 'width_over_height': value[1], 'label': 'closed'}

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8774 entries, 0 to 8773
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   normalized_sclera_area  8774 non-null   float64
 1   width_over_height       8774 non-null   float64
 2   label                   8774 non-null   object 
dtypes: float64(2), object(1)
memory usage: 274.2+ KB


In [78]:
df

,normalized_sclera_area,width_over_height,label
0,0.337145,2.049031,wide
1,0.242546,2.705165,wide
2,0.303419,2.214134,wide
3,0.306884,2.145044,wide
4,0.283548,2.275349,wide
...,...,...,...
8769,0.004560,20.000000,closed
8770,0.004815,5.608587,closed
8771,0.011204,4.744976,closed
8772,0.059252,9.084570,closed


In [79]:
df.to_json('data.json', index=False)